In [1]:
import importlib
import pandas as pd

import utils
from usgs_querier import EQFromUSGS

importlib.reload(utils)


<module 'utils' from '/Users/qingkaikong/Desktop/USGS/notebooks/utils.py'>

In [2]:
df_eq = pd.read_pickle('../data/USGS_Earthquake_20160201_20200801_M3.5above.pkl')
df_event = pd.read_hdf('../data/event_20160201_20190929.h5')
df_matched_event = pd.read_csv('../data/matched_events_from_SA_20160201_20201006.csv')

In [5]:
df_matched_event[df_matched_event['catalogid'] == 'nc72721105']

,Unnamed: 0,id,ver,catid,time,day,dsecs,dkm,dmag,modtime,id.1,catalogid,lat,lon,depth,mag,time.1,day.1,modtime.1
10368,160049,26034,0,285474,1.478437e+09,20161106,2.652,6.08058,0.0429,1478437913,285474,nc72721105,39.659833,-123.637333,-0.65,4.06,1.478437e+09,20161106,1478577633
10369,160050,26033,0,285474,1.478437e+09,20161106,2.652,6.08058,0.0429,1478437913,285474,nc72721105,39.659833,-123.637333,-0.65,4.06,1.478437e+09,20161106,1478577633
10370,160051,26032,0,285474,1.478437e+09,20161106,2.652,6.08058,0.0429,1478437913,285474,nc72721105,39.659833,-123.637333,-0.65,4.06,1.478437e+09,20161106,1478577633
10371,160052,26031,0,285474,1.478437e+09,20161106,2.652,6.08058,0.0429,1478437913,285474,nc72721105,39.659833,-123.637333,-0.65,4.06,1.478437e+09,20161106,1478577633
10372,160053,26030,0,285474,1.478437e+09,20161106,4.577,22.48430,-0.4484,1478437913,285474,nc72721105,39.659833,-123.637333,-0.65,4.06,1.478437e+09,20161106,1478577633


In [82]:
len(df_eq)

788

## Get metrics

* We will get the following metrics:
    * True positive - SA sent out alert for the USGS event. 
    * False positive - SA sent out alert but no corresponding USGS event. 
    * False negative - SA didn't send out an alert for USGS event. 
    
#### Get True positive and False negative

In [123]:
false_negative_list = []
true_positive_list = []

for ix, row in df_eq.iterrows():
    evid = row['id']
    
    if evid in df_matched_event.catalogid.to_list():
        true_positive_list.append(evid)
    else:
        false_negative_list.append(evid)

In [126]:
df_event_select = df_event[df_event['system']=='dm']
df_event_select = df_event_select[df_event_select['type']=='new']

In [124]:
len(true_positive_list)

603

In [125]:
    len(false_negative_list)

185

#### Get false positive

In [128]:
mag_threshold = 3.5

# We get the first alert of all the ShakeAlert events.
df_event_select = df_event[(df_event['system']=='dm') & (df_event['type']=='new')]
df_event_select_m3pt5 = df_event_select[df_event_select['mag'] >= mag_threshold]

In [129]:
false_positive_list = []

for _, row in df_event_select_m3pt5.iterrows():
    
    if row.id not in df_matched_event['id'].to_list():
        false_positive_list.append(row.id)

In [130]:
len(false_positive_list)

64

## Now let's see how many of these events we can help

#### Let's look at the 185 False Negative Events, how many of them can we help

In [33]:
import glob
import pickle

In [ ]:
def get_event_number_for_syn_triggers_before_time(simulation_dict, 
                                                  alert_time,
                                                  evla,
                                                  evlo):
    steady_phone_stats, trig_stats = \
            utils.simulation_data_count(simulation_dict,
                                        alert_time,
                                        evla,
                                        evlo)
    
    count_more_than_10 = 0
    count_more_than_50 = 0
    count_more_than_100 = 0
    count_more_than_500 = 0
    
    if trig_stats['median'] > 10:
        count_more_than_10 = 1

    if trig_stats['median'] > 50:
        count_more_than_50 = 1

    if trig_stats['median'] > 100:
        count_more_than_100 = 1

    if trig_stats['median'] > 100:
        count_more_than_500 = 1
            
    return count_more_than_10, count_more_than_50, count_more_than_100, count_more_than_500

In [170]:
count_more_than_10_5s = 0
count_more_than_50_5s = 0
count_more_than_100_5s = 0
count_more_than_500_5s = 0

count_more_than_10_10s = 0
count_more_than_50_10s = 0
count_more_than_100_10s = 0
count_more_than_500_10s = 0

count_more_than_10_15s = 0
count_more_than_50_15s = 0
count_more_than_100_15s = 0
count_more_than_500_15s = 0

count_more_than_10_20s = 0
count_more_than_50_20s = 0
count_more_than_100_20s = 0
count_more_than_500_20s = 0

count_more_than_10_30s = 0
count_more_than_50_30s = 0
count_more_than_100_30s = 0
count_more_than_500_30s = 0

for i, evid in enumerate(false_negative_list):
    if i%10==0:
        print(i)
        
    filename = glob.glob(f'../data/simulation_data/*{evid}*')
    earthquake_file = glob.glob(f'../data/ground_truth/*{evid}*')
    
    if len(earthquake_file)>0:
        earthquake = pd.read_pickle(earthquake_file[0])
    else:
        print(f'No data for {evid}')
        continue
    evla = earthquake['latitude']
    evlo = earthquake['longitude']
    
    if len(filename) > 0:
        filename = filename[0]
    else:
        continue
        
    simulation_dict = pickle.load(open(filename, 'rb'))
    
    # check how many event contain triggers within 5s after the event 
    n10, n50, n100, n500 = \
            get_event_number_for_syn_triggers_before_time(simulation_dict, 
                                                          5,
                                                          evla,
                                                          evlo)

    count_more_than_10_5s += n10
    count_more_than_50_5s += n50
    count_more_than_100_5s += n100
    count_more_than_500_5s += n500
    
    # check how many event contain triggers within 10s after the event 
    n10, n50, n100, n500 = \
            get_event_number_for_syn_triggers_before_time(simulation_dict, 
                                                          10,
                                                          evla,
                                                          evlo)

    count_more_than_10_10s += n10
    count_more_than_50_10s += n50
    count_more_than_100_10s += n100
    count_more_than_500_10s += n500
    
    # check how many event contain triggers within 15s after the event 
    n10, n50, n100, n500 = \
            get_event_number_for_syn_triggers_before_time(simulation_dict, 
                                                          15,
                                                          evla,
                                                          evlo)

    count_more_than_10_15s += n10
    count_more_than_50_15s += n50
    count_more_than_100_15s += n100
    count_more_than_500_15s += n500
    
    # check how many event contain triggers within 20s after the event 
    n10, n50, n100, n500 = \
            get_event_number_for_syn_triggers_before_time(simulation_dict, 
                                                          20,
                                                          evla,
                                                          evlo)

    count_more_than_10_20s += n10
    count_more_than_50_20s += n50
    count_more_than_100_20s += n100
    count_more_than_500_20s += n500       
    
    # check how many event contain triggers within 30s after the event 
    n10, n50, n100, n500 = \
            get_event_number_for_syn_triggers_before_time(simulation_dict, 
                                                          30,
                                                          evla,
                                                          evlo)

    count_more_than_10_30s += n10
    count_more_than_50_30s += n50
    count_more_than_100_30s += n100
    count_more_than_500_30s += n500   
    

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180


In [166]:
print(count_more_than_10_5s,
      count_more_than_50_5s,
      count_more_than_100_5s,
      count_more_than_500_5s)

1 0 0 0


In [167]:
print(count_more_than_10_10s,
      count_more_than_50_10s,
      count_more_than_100_10s,
      count_more_than_500_10s)

6 0 0 0


In [168]:
print(count_more_than_10_15s,
      count_more_than_50_15s,
      count_more_than_100_15s,
      count_more_than_500_15s)

7 0 0 0


In [169]:
print(count_more_than_10_20s,
      count_more_than_50_20s,
      count_more_than_100_20s,
      count_more_than_500_20s)

8 0 0 0


In [171]:
print(count_more_than_10_30s,
      count_more_than_50_30s,
      count_more_than_100_30s,
      count_more_than_500_30s)

65 0 0 0


### How many could be faster

For all the detected events, how many of them we can potentially speed up the whole process. 

In [147]:
all_files = glob.glob("./count_*")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

df_count_results = pd.concat(li, axis=0, ignore_index=True)


In [148]:
len(true_positive_list)

603

In [150]:
# There are some matched alerts has negative alert time,
# maybe matched the wrong event?
events_has_negative_alert_time = []
count_more_than_10_first_alert = 0
count_more_than_50_first_alert = 0
count_more_than_100_first_alert = 0
count_more_than_500_first_alert = 0

count_more_than_10_half_first_alert = 0
count_more_than_50_half_first_alert = 0
count_more_than_100_half_first_alert = 0
count_more_than_500_half_first_alert = 0

for evid in true_positive_list:
    
    df_matched_event_select = df_matched_event[df_matched_event['catalogid'] == evid]
    evtime = df_matched_event_select['time.1'].iloc[0]
    evla = df_matched_event_select.iloc[0]['lat']
    evlo = df_matched_event_select.iloc[0]['lon']
    df_matched_event_select = df_matched_event_select.sort_values('time')
    
    for ix, row in df_matched_event_select.iterrows():
        event_id = row['id']
        
        df_alert = df_event_select[df_event_select['id'] == event_id]
        min_alert_time = 9999
        if len(df_alert) > 0:
            alert_time_s = (df_alert['alert_time'] - evtime).values[0]
            if (alert_time_s < min_alert_time) & (alert_time_s > 0):
                alert_info = df_alert
                first_alert_time = alert_time_s
                min_alert_time = alert_time_s
        else:
            alert_info = None
            first_alert_time = -9999
    
    if alert_info is None:
        events_has_negative_alert_time.append(evid)
    else:
        # We count how many triggers do we have
        filename = glob.glob(f'../data/simulation_data/*{evid}*')
    
        if len(filename) > 0:
            filename = filename[0]
        else:
            continue

        simulation_dict = pickle.load(open(filename, 'rb'))
        
        # count the event has triggers before the first alert time
        n10, n50, n100, n500 = \
            get_event_number_for_syn_triggers_before_time(simulation_dict, 
                                                          first_alert_time,
                                                          evla,
                                                          evlo)
        
        count_more_than_10_first_alert += n10
        count_more_than_50_first_alert += n50
        count_more_than_100_first_alert += n100
        count_more_than_500_first_alert += n500
        
        # # count the event has triggers before half of the the first alert time
        n10, n50, n100, n500 = \
            get_event_number_for_syn_triggers_before_time(simulation_dict, 
                                                          first_alert_time/2.0,
                                                          evla,
                                                          evlo)
        
        count_more_than_10_half_first_alert += n10
        count_more_than_50_half_first_alert += n50
        count_more_than_100_half_first_alert += n100
        count_more_than_500_half_first_alert += n500

In [151]:
print(count_more_than_10_first_alert,
      count_more_than_50_first_alert,
      count_more_than_100_first_alert,
      count_more_than_500_first_alert)

136 51 30 30


In [152]:
print(count_more_than_10_half_first_alert,
      count_more_than_50_half_first_alert,
      count_more_than_100_half_first_alert,
      count_more_than_500_half_first_alert)

45 16 9 9


In [153]:
events_has_negative_alert_time

['nc71103759',
 'ci37374711',
 'nc72974766',
 'ci38238754',
 'ci37421757',
 'ci38462063',
 'ci38461007',
 'ci38459335',
 'ci38458967',
 'ci38457991',
 'ci38457863',
 'ci38457855',
 'ci38457847',
 'ci37420717',
 'ci38443543',
 'ci38443439',
 'ci38443391']